In [1]:
# Import necessary libraries
import os
import django
import pandas as pd
import json

In [2]:
# Set up Django environment
os.chdir("..")
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'gymcrowd.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()


In [3]:
# Import models
from apps.workouts.models import Exercise


In [6]:
# Load JSON data into a pandas DataFrame
json_file_path = 'scripts/exercises.json'

with open(json_file_path, 'r') as f:
    exercise_data = json.load(f)

# Convert JSON data to pandas DataFrame
df = pd.DataFrame(exercise_data)
print(f"Loaded {len(df)} exercises from the JSON file.")
df.head()  # Display the first few rows for verification


Loaded 1324 exercises from the JSON file.


,bodyPart,equipment,gifUrl,id,name,target,secondaryMuscles,instructions
0,waist,body weight,https://v2.exercisedb.io/image/ogjzsJ3u4sUljM,0001,3/4 sit-up,abs,"[hip flexors, lower back]",[Lie flat on your back with your knees bent an...
1,waist,body weight,https://v2.exercisedb.io/image/Lg7bSGP4-8PzGd,0002,45° side bend,abs,[obliques],[Stand with your feet shoulder-width apart and...
2,waist,body weight,https://v2.exercisedb.io/image/Izy-UV0PspQK87,0003,air bike,abs,[hip flexors],[Lie flat on your back with your hands placed ...
3,upper legs,body weight,https://v2.exercisedb.io/image/umGVOCitxmf4mP,1512,all fours squad stretch,quads,"[hamstrings, glutes]",[Start on all fours with your hands directly u...
4,waist,body weight,https://v2.exercisedb.io/image/7XA7D-w2bmpgsk,0006,alternate heel touchers,abs,[obliques],[Lie flat on your back with your knees bent an...


In [7]:
# Transform data for secondary muscles and instructions
df['secondaryMuscles'] = df['secondaryMuscles'].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")
df['instructions'] = df['instructions'].apply(lambda x: " ".join(x) if isinstance(x, list) else "")
df.head()  # Check transformed data


,bodyPart,equipment,gifUrl,id,name,target,secondaryMuscles,instructions
0,waist,body weight,https://v2.exercisedb.io/image/ogjzsJ3u4sUljM,0001,3/4 sit-up,abs,"hip flexors, lower back",Lie flat on your back with your knees bent and...
1,waist,body weight,https://v2.exercisedb.io/image/Lg7bSGP4-8PzGd,0002,45° side bend,abs,obliques,Stand with your feet shoulder-width apart and ...
2,waist,body weight,https://v2.exercisedb.io/image/Izy-UV0PspQK87,0003,air bike,abs,hip flexors,Lie flat on your back with your hands placed b...
3,upper legs,body weight,https://v2.exercisedb.io/image/umGVOCitxmf4mP,1512,all fours squad stretch,quads,"hamstrings, glutes",Start on all fours with your hands directly un...
4,waist,body weight,https://v2.exercisedb.io/image/7XA7D-w2bmpgsk,0006,alternate heel touchers,abs,obliques,Lie flat on your back with your knees bent and...


In [8]:
# Fetch existing exercises from the database
existing_exercises = set(Exercise.objects.values_list('name', flat=True))

# Split data into new and existing records
df_existing = df[df['name'].isin(existing_exercises)]
df_new = df[~df['name'].isin(existing_exercises)]

print(f"Existing exercises: {len(df_existing)}")
print(f"New exercises: {len(df_new)}")


Existing exercises: 0
New exercises: 1324


In [9]:
# Update existing records
for _, row in df_existing.iterrows():
    Exercise.objects.filter(name=row['name']).update(
        body_part=row['bodyPart'],
        equipment=row['equipment'],
        gif_url=row['gifUrl'],
        target=row['target'],
        secondary_muscles=row['secondaryMuscles'],
        instructions=row['instructions'],
    )
print("Updated existing records.")


Updated existing records.


In [10]:
# Bulk insert new records
new_records = [
    Exercise(
        name=row['name'],
        body_part=row['bodyPart'],
        equipment=row['equipment'],
        gif_url=row['gifUrl'],
        target=row['target'],
        secondary_muscles=row['secondaryMuscles'],
        instructions=row['instructions'],
    )
    for _, row in df_new.iterrows()
]
Exercise.objects.bulk_create(new_records, batch_size=500)

print(f"Inserted {len(new_records)} new records.")


Inserted 1324 new records.


In [11]:
# Verify the database content
total_exercises = Exercise.objects.count()
print(f"Total exercises in the database: {total_exercises}")

# Display some records from the database
Exercise.objects.all()[:5]  # Display the first 5 records


Total exercises in the database: 1324


<QuerySet [<Exercise: 3/4 sit-up>, <Exercise: 45° side bend>, <Exercise: air bike>, <Exercise: all fours squad stretch>, <Exercise: alternate heel touchers>]>